# RAG

This notebook introduces how to build simple RAG application

In [ ]:
import boto3, json, textwrap
import re
from enum import Enum
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# Create a session with AWS and initialize a Bedrock client for conversational AI models
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')


In [ ]:

class Models(Enum):
    # Enum for storing model identifiers for different AI models.
    # Each member represents a specific model hosted by various platforms like Anthropic and Meta.
    
    Sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"  
    Haiku = "anthropic.claude-3-haiku-20240307-v1:0"   
    Llama = "meta.llama3-8b-instruct-v1:0"            
    Cohere = "cohere.command-r-plus-v1:0"      

In [ ]:
# List to hold the conversation's flow
message_list = []

# Starting message from the user to the model
initial_message = {
    "role": "user",
    "content": [
        { "text": "How are you today?" } 
    ],
}

# Append the initial message to the conversation list
message_list.append(initial_message)


In [ ]:
# Using Bedrock SDK to send a message to the model and get a response
response = bedrock.converse(
    modelId=Models.Sonnet.value,
    messages=message_list,  # Conversation history
    inferenceConfig={
        "maxTokens": 3000,  # Maximum length of the model's response
        "temperature": 0    # Sampling temperature, 0 for deterministic response
    },
)


In [ ]:
# Extract the message part of the response from the model
response_message = response['output']['message']

# Print the response message formatted as JSON for readability
print(json.dumps(response_message, indent=4))


In [ ]:
def get_vectors(text):
    response = bedrock.invoke_model(
        body=json.dumps({
            'inputText': text
        }),
        modelId="amazon.titan-embed-text-v1",
        accept="application/json",
        contentType="application/json",
    )
    #print(response)
    response_body = json.loads(response.get("body").read())
    return response_body.get("embedding")

In [ ]:
def get_oss_connection():
    host = 'erz3fznnf3antezuu2bc.us-west-2.aoss.amazonaws.com'
    region = 'us-west-2'
    service = 'aoss'
    index = 'bedrock-knowledge-base-default-index'
    credentials = boto3.Session().get_credentials()
    auth = AWSV4SignerAuth(credentials, region, service)

    ospy_client = OpenSearch(
        hosts = [{'host': host, 'port': 443}],
        http_auth = auth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection,
        pool_maxsize = 20
    )

    return ospy_client

In [ ]:
question = 'What are the characteristics of bull dog breed?'
vectors = get_vectors(question)
k = 4 # number of neighbours, size and k are the same to return k results in total. If size is not specified, k results will be returned per shard.
qry = {
  "size": 3,
  "query": {
    "knn": {
      "bedrock-knowledge-base-default-vector": {
        "vector": vectors,
        "k": 2
      }
    }
  }
}

print("Query generated for OpenSearch")
print(json.dumps(qry, indent=4))
print("================================")

In [ ]:

oss_client = get_oss_connection()

index = 'bedrock-knowledge-base-default-index'

query = {
    "size": 7,
    "query": {
        "knn": {
        "bedrock-knowledge-base-default-vector": {
            "vector": vectors,
            "k": 2
        }
        }
    }
    }

response = oss_client.search(
        body = query,
        index = index
    )

hits = response['hits']['hits']
context = []
for hit in hits:
        #print(json.dumps(hit["_source"]["AMAZON_BEDROCK_TEXT_CHUNK"], indent=4))
    context.append(hit["_source"]["AMAZON_BEDROCK_METADATA"])
    context.append(hit["_source"]["AMAZON_BEDROCK_TEXT_CHUNK"])

print("Response from OpenSearch")
print(json.dumps(context, indent=4))


In [ ]:
prompt_template = """You are a friendly and honest chatbot agent.
        Based on the contents of <doc/> XML tag, provide answer including document source for, question listed in <question/> XML tag. 
        Answer "Error: I did not find a valid answer." if not present in the document. 
        <doc>{context}</doc>
        <question>{question}</question>"""


llm_prompt = prompt_template.format(context='\n'.join(context),question=question)

    # List to hold the conversation's flow
message_list = []

    # Starting message from the user to the model
initial_message = {
        "role": "user",
        "content": [
            { "text": llm_prompt } 
        ],
    }

    # Append the initial message to the conversation list
message_list.append(initial_message)

response = bedrock.converse(
        modelId=Models.Sonnet.value,
        messages=message_list,  # Conversation history
        inferenceConfig={
            "maxTokens": 2000,  # Maximum length of the model's response
            "temperature": 0    # Sampling temperature, 0 for deterministic response
        },
    )

print(json.dumps(response, indent=4))